In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ExponentialLR

In [2]:
# Get CPU or GPU device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)

In [3]:
# Random seed for reproducibility
seed = 5
torch.manual_seed(seed)

In [4]:
# Save the model at the end?
save_model = False

In [5]:
# Batch sizes for training and testing
batch_size = 64
test_batch_size = 14

In [6]:
# Training epochs (usually set to 10)
n_epochs = 10

# Learning rate
learning_rate = 1.0

# Decay rate for adjusting the learning rate
gamma = 0.7

# How many batches before logging training status
log_interval = 10

# Number of target classes in the MNIST data
num_classes = 10

In [7]:
train_kwargs = {"batch_size": batch_size}
test_kwargs = {"batch_size": test_batch_size}

# CUDA settings
if torch.cuda.is_available():
    cuda_kwargs = {"num_workers": 1, "pin_memory": True, "shuffle": True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)


# The scaled mean and standard deviation of the MNIST dataset (precalculated)
data_mean = 0.1307
data_std = 0.3081

# Convert input images to tensors and normalize
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((data_mean,), (data_std,))]
)

In [8]:
# Get the MNIST data from torchvision
dataset1 = datasets.MNIST("../data", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("../data", train=False, transform=transform)

# Define the data loaders that will handle fetching of data
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [9]:
# Define the architecture of the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding="valid")
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding="valid")
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # sum up batch loss
            test_loss += F.nll_loss(output, target, reduction="sum").item()
            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )


In [ ]:
# Send the model to the device (CPU or GPU)
model = Net().to(device)

# Define the optimizer to user for gradient descent
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)

# Shrinks the learning rate by gamma every step_size
scheduler = ExponentialLR(optimizer, gamma=gamma)

# Train the model
for epoch in range(1, n_epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)
    scheduler.step()



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304271
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.047250
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.078645
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.754417
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.709935
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.613494
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.622623
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.637053
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.704263
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.577765
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.585352
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.535475
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.564499
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.531173
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.540764
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.587100
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.744055
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.559352
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.608556
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.587780
Train Epoch: 1 [12800/60000 (